In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os

In [2]:

# Hyperparameters
lookback_window = 7  # Number of time steps
#I don't have any idea which number is best
# from past test it looks like model has an understanding of general trend of the stock
# but not the specific chracteristics of the stock
# chatgpt recommends me to increase lookback window
input_size = 6        # Number of features (univariate = 1)
hidden_size = 50      # Number of LSTM neurons
num_layers = 2        # Number of LSTM layers
output_size = 1       # Predicting one value
learning_rate = 0.001
epochs = 50
batch_size = 32


In [3]:
dataLocation="../Data/StandardizedData"
allcsvs=os.listdir(dataLocation)
# turn float64 to float 32

In [ ]:
# changeMatrix=rawData.copy()
# changeMatrix["Open"]=changeMatrix["Open"].pct_change()
# changeMatrix["High"]=changeMatrix["High"].pct_change()
# changeMatrix["Low"]=changeMatrix["Low"].pct_change()
# changeMatrix["Close"]=changeMatrix["Close"].pct_change()
# changeMatrix["Volume"]=changeMatrix["Volume"].pct_change()
# # changeMatrix.dropna(inplace=True)
# changeMatrix.fillna(0,inplace=True)
# # changeMatrix.drop(columns=["Pct_Change"],inplace=True)
# changeMatrix

In [4]:
unnecessaryColumns=["Date","Pct_Change"]

from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()


def df_to_X_y(df, scaler ,window_size=5):
  lightWeightDf=df.drop(columns=unnecessaryColumns)
  df_as_np = scaler.fit_transform(lightWeightDf.to_numpy())
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [a for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size,3]
    y.append(label)
  return np.array(X), np.array(y)

In [23]:
# XAll = np.zeros(1, dtype=np.int32)
# yAll = np.zeros(1, dtype=np.int32)

# for i in range(len(allcsvs)):
#     rawData=pd.read_csv(os.path.join(dataLocation,allcsvs[i]))
#     Xtemp,yTemp=df_to_X_y(rawData, scaler ,lookback_window)
#     if(XAll.shape[0] == 1 and yAll.shape[0] == 1):
#         XAll=Xtemp
#         yAll=yTemp
#     else:
#         XAll=np.concatenate((XAll,Xtemp),axis=0)
#         yAll=np.concatenate((yAll,yTemp),axis=0)


# I have to store all of the rawData accurately because I need them afterwards
# or I can make a huge tensor and store all of them in it
# but this raises concerns about the time series consistency

In [24]:
# XAll.shape, yAll.shape

((57412, 7, 6), (57412,))

In [5]:
def shuffle_data(X, y):
    """
    Shuffle the data while maintaining the correspondence between X and y.

    Parameters:
    X (numpy.ndarray): The input features tensor.
    y (numpy.ndarray): The target array.

    Returns:
    tuple: Shuffled X and y tensors.
    """
    assert len(X) == len(y), "X and y must have the same length."
    
    # Generate a random permutation of indices
    indices = np.random.permutation(len(X))
    # print(indices[:5])
    # Shuffle X and y
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    return X_shuffled, y_shuffled

In [29]:
# X_shuffled, y_shuffled = shuffle_data(XAll, yAll)

[26684  1127 52441 48644 36731]


In [6]:
class TimeSeriesDataset(Dataset):
    def __init__(self, allcsvLoc : list, dataDir):
        self.scaler =MinMaxScaler()
        X = np.zeros(1)
        y = np.zeros(1)

        for i in range(len(allcsvLoc)):
            rawData=pd.read_csv(os.path.join(dataDir,allcsvLoc[i]))
            Xtemp,yTemp=df_to_X_y(rawData, self.scaler, lookback_window)
            if(X.shape[0] == 1 and y.shape[0] == 1):
                X=Xtemp
                y=yTemp
            else:
                X=np.concatenate((X,Xtemp),axis=0)
                y=np.concatenate((y,yTemp),axis=0)


        #shuffle the data
        #NOTE shuffling is automatically done in the dataloader
        X_shuffled, y_shuffled = shuffle_data(X, y)
        self.dataX = torch.tensor(X_shuffled, dtype=torch.float32)
        self.dataY = torch.tensor(y_shuffled, dtype=torch.float32)
        
        

    def __len__(self):
        return self.dataX.size(0)

    def __getitem__(self, idx):
        sequence = self.dataX[idx, :]
        target = self.dataY[idx]
        return sequence, target

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x)
        last_output = output[:, -1, :]  # Take the output at the last time step
        return self.fc(last_output)
    
    # def init_weights(module):
    #     if isinstance(module, nn.LSTM):
    #         nn.init.xavier_normal_(module.weight_ih_l0)
    #         nn.init.xavier_normal_(module.weight_hh_l0)
    #         if module.is_bidirectional:
    #             nn.init.xavier_normal_(module.weight_ih_l0_reverse)
    #             nn.init.xavier_normal_(module.weight_hh_l0_reverse) 


In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using {device}")

Using cuda


In [ ]:
X_train = X_shuffled[:int(0.8*len(X_shuffled))]
y_train = y_shuffled[:int(0.8*len(y_shuffled))]
X_val = X_shuffled[int(0.8*len(X_shuffled)):]
y_val = y_shuffled[int(0.8*len(y_shuffled)):]

In [ ]:
trainDataset = TimeSeriesDataset(rawData)
train_loader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
# If you want to maintain proper batching while avoiding overfitting to a specific order during training, you could use techniques like:

# Random Sampling of Starting Points: Without fully shuffling the data.
# Cross-validation: On non-overlapping sequential splits for evaluation.


# Initialize model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for sequences, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

Epoch 1/50, Loss: 0.0510
Epoch 2/50, Loss: 0.0026
Epoch 3/50, Loss: 0.0011
Epoch 4/50, Loss: 0.0009
Epoch 5/50, Loss: 0.0010
Epoch 6/50, Loss: 0.0010
Epoch 7/50, Loss: 0.0009
Epoch 8/50, Loss: 0.0008
Epoch 9/50, Loss: 0.0009
Epoch 10/50, Loss: 0.0008
Epoch 11/50, Loss: 0.0008
Epoch 12/50, Loss: 0.0008
Epoch 13/50, Loss: 0.0007
Epoch 14/50, Loss: 0.0007
Epoch 15/50, Loss: 0.0008
Epoch 16/50, Loss: 0.0007
Epoch 17/50, Loss: 0.0007
Epoch 18/50, Loss: 0.0007
Epoch 19/50, Loss: 0.0008
Epoch 20/50, Loss: 0.0008
Epoch 21/50, Loss: 0.0006
Epoch 22/50, Loss: 0.0007
Epoch 23/50, Loss: 0.0006
Epoch 24/50, Loss: 0.0006
Epoch 25/50, Loss: 0.0006
Epoch 26/50, Loss: 0.0006
Epoch 27/50, Loss: 0.0005
Epoch 28/50, Loss: 0.0006
Epoch 29/50, Loss: 0.0005
Epoch 30/50, Loss: 0.0005
Epoch 31/50, Loss: 0.0006
Epoch 32/50, Loss: 0.0005
Epoch 33/50, Loss: 0.0005
Epoch 34/50, Loss: 0.0005
Epoch 35/50, Loss: 0.0005
Epoch 36/50, Loss: 0.0005
Epoch 37/50, Loss: 0.0005
Epoch 38/50, Loss: 0.0005
Epoch 39/50, Loss: 0.

In [34]:
# Extract one batch of data for testing
for sequences, targets in train_loader:
    testInput = sequences  # Sequence data
    print(f"Test Input Shape: {testInput.shape}")
    realOutput = targets   # Ground truth output
    break

# Ensure the model is in evaluation mode
model.eval()

# Predict using the model
with torch.no_grad():  # Disable gradient calculations for inference
    predicted = model(testInput).squeeze()  # Forward pass through the model

# Check shapes and results
print(f"Predicted Output Shape: {predicted.shape}")
print(f"First Prediction: {predicted[0].item()}")
print(f"Actual Output (First Target): {realOutput[0].item()}")

predicted
# testInput.shape

Test Input Shape: torch.Size([32, 7, 6])
Predicted Output Shape: torch.Size([32])
First Prediction: 0.8080300092697144
Actual Output (First Target): 0.8317721486091614


tensor([0.8080, 0.0297, 0.1301, 0.5177, 0.3785, 0.1505, 0.8001, 0.5566, 0.1417,
        0.1639, 0.0841, 0.1765, 0.0565, 0.3737, 0.0722, 0.5325, 0.2416, 0.1002,
        0.3656, 0.1389, 0.0771, 0.5591, 0.1352, 0.0634, 0.1092, 0.0532, 0.1175,
        0.1679, 0.5604, 0.3900, 0.6289, 0.0728])

In [35]:
realOutput

tensor([0.8318, 0.0333, 0.1348, 0.5432, 0.3610, 0.1396, 0.7947, 0.5679, 0.1442,
        0.1689, 0.0879, 0.1788, 0.0589, 0.3629, 0.0802, 0.5601, 0.2882, 0.1125,
        0.3657, 0.1365, 0.0835, 0.5416, 0.1391, 0.0667, 0.1050, 0.0495, 0.1125,
        0.1783, 0.5864, 0.4106, 0.6423, 0.0760])